<a href="https://colab.research.google.com/github/dantae74/web-crawling/blob/main/5_%EB%8F%99%EC%A0%81_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 동적 데이터 수집

---

## 0. 라이브러리 import

In [4]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.dUuKpAn2UZ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.KCV0kQOBKy/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.xEVMvrYjJx/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [5]:
!wget https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!chmod 777 chromedriver
!mv chromedriver /usr/bin/

--2023-06-30 06:53:27--  https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 142.251.171.128, 2607:f8b0:4001:c5f::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|142.251.171.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5795569 (5.5M) [application/zip]
Saving to: ‘chromedriver_linux64.zip’

chromedriver_linux6 100%[===================>]   5.53M  --.-KB/s    in 0.03s   

2023-06-30 06:53:27 (173 MB/s) - ‘chromedriver_linux64.zip’ saved [5795569/5795569]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

url = "http://example.com"

service = Service("/usr/bin/chromedriver")

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=service, options=options)

driver.get(url)
print(driver.title)
driver.quit()

Example Domain


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

import time

## 1. 웹드라이버 다루기

In [7]:
# 드라이버 로드
# driver = webdriver.Chrome()
driver = webdriver.Chrome(service=service, options=options)

# 페이지 접근
driver.get('http://quotes.toscrape.com/') # 크롤링 예제 사이트

# 드라이버 종료
# driver.close()

In [ ]:
# 드라이버에서 요소 찾기

## 태그 이름으로 찾기
# driver.find_element(By.TAG_NAME, 'TAG_NAME')

## 아이디로 찾기
# driver.find_element(By.ID, 'ID')

## 클래스 이름으로 찾기
# driver.find_element(By.CLASS_NAME, 'CLASS_NAME')

## css 셀렉터로 찾기
# driver.find_element(By.CSS_SELECTOR, 'CSS_SELECTOR')

# 드라이버에서 여러 요소 찾기
# driver.find_elements(By.CSS_SELECTOR, 'CSS_SELECTOR')

- 만약 데이터를 구하기 위해 상호작용 필요하다면 키 입력이나, 클릭 등을 자동화해야 한다.

### 로그인하기

In [11]:
# 로그인 버튼 찾기
login_btn = driver.find_element(By.CSS_SELECTOR, 'body > div > div.row.header-box > div.col-md-4 > p > a')


In [12]:
# 로그인 버튼 클릭
login_btn.click()

In [13]:
# id, pw 입력
id_input = driver.find_element(By.CSS_SELECTOR, '#username')
password_input = driver.find_element(By.CSS_SELECTOR, '#password')

In [14]:
id_input.send_keys('test')
password_input.send_keys('test')

In [15]:
# login 버튼 클릭
login_btn = driver.find_element(By.CSS_SELECTOR, 'body > div > form > input.btn.btn-primary')
login_btn.click()

### 데이터 추출하기

In [ ]:
# 요소에서 텍스트 추출하는 방법
# 요소.text

# 요소에서 속성 추출하는 방법
# 요소.get_attribute('속성명')

In [16]:
# 인용구 리스트 추출
# body > div > div:nth-child(2) > div.col-md-8 > div:nth-child(1)
# 마지막 div에서 순서를 나타내는 nth-child(1)을 제거하면 전체 리스트를 가져올 수 있음

quotes = driver.find_elements(By.CSS_SELECTOR, 'body > div > div:nth-child(2) > div.col-md-8 > div')
print(len(quotes))

10


In [17]:
for quote in quotes:
    print("인용문 : "+quote.find_element(By.CSS_SELECTOR, 'span:nth-of-type(1)').text)
    print("인물 : "+quote.find_element(By.CSS_SELECTOR, 'span:nth-of-type(2) > small').text)
    print("인물 정보 링크 : "+quote.find_element(By.CSS_SELECTOR, 'span:nth-of-type(2) > a:nth-of-type(1)').get_attribute('href'))
    print('----------------------------------')


인용문 : “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
인물 : Albert Einstein
인물 정보 링크 : http://quotes.toscrape.com/author/Albert-Einstein
----------------------------------
인용문 : “It is our choices, Harry, that show what we truly are, far more than our abilities.”
인물 : J.K. Rowling
인물 정보 링크 : http://quotes.toscrape.com/author/J-K-Rowling
----------------------------------
인용문 : “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
인물 : Albert Einstein
인물 정보 링크 : http://quotes.toscrape.com/author/Albert-Einstein
----------------------------------
인용문 : “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
인물 : Jane Austen
인물 정보 링크 : http://quotes.toscrape.com/author/Jane-Austen
----------------------------------
인용문 : “Imperfection is beauty, madness is genius and it's better to be absolutely ri

In [18]:
driver.close()

---

### element 로딩을 기다리기

In [20]:
# 드라이버 로드
# driver = webdriver.Chrome()
driver = webdriver.Chrome(service=service, options=options)

# 페이지 접근
driver.get('https://map.naver.com/') # 크롤링 예제 사이트

# 검색 창에 검색어 입력
search_input = driver.find_element(By.CSS_SELECTOR, 'input[id^="input_search"]')    # id 가 새롭게 로드될 때 마다 바뀌는 구조
                                                                                    # input 태그의 id 속성이 input_search로 시작하는 요소를 찾아라

search_input.send_keys('대구광역시 북구 고성동 치킨')


NoSuchElementException: ignored

- 왜 에러가 발생하는 걸까요?

In [21]:
# 다시 실행하면 엘리먼트가 찾아짐
search_input = driver.find_element(By.CSS_SELECTOR, 'input[id^="input_search"]')
search_input.send_keys('대구광역시 북구 고성동 치킨')

- 검색창이 로딩되기 전에 탐색을 시도하여 없다고 판단하고 에러를 발생시킴
- 이럴땐 찾으려는 요소가 나타날때 까지 기다려줘야한다

In [22]:
## 요소를 기다리는 코드 추가

# 드라이버 로드
# driver = webdriver.Chrome()
driver = webdriver.Chrome(service=service, options=options)

# 페이지 접근
driver.get('https://map.naver.com/') # 크롤링 예제 사이트

# 검색 element 가 로드될 때까지 최대 10초간 대기
search_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[id^="input_search"]'))) # 요소가 나타날때 까지 최대 10초 대기

# 검색 창에 검색어 입력
search_input.send_keys('대구광역시 북구 고성동 치킨')

# enter 입력
search_input.send_keys(Keys.ENTER)

- wait 조건이 나열되어 있는 문서

https://selenium-python.readthedocs.io/waits.html